모듈 ∙ 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
# tf.config.experimental.list_physical_devices('GPU')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

데이터 불러오기 ∙ 전처리

In [2]:
def filter_df(df): #데이터프레임 필터링
    # 첫 번째 열에서 같은 값을 가진 행의 수를 계산합니다.
    row_counts = df['0'].value_counts()

    # 가장 많은 행의 수를 찾습니다.
    max_row_count = row_counts.max()

    # 가장 많은 행의 수에 해당하는 행만 분류합니다.
    filtered = pd.DataFrame(df[df['0'].isin(row_counts[row_counts == max_row_count].index)])

    return filtered

# 데이터 로드   
stock_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_5%.csv',low_memory=False)

#데이터 필터링
filter_stock = filter_df(stock_df)
filter_label = filter_stock['24']

# 불필요한 데이터 삭제
filter_stock = filter_stock.drop({'0','1','7','24'},axis=1) #날자, 상승율, 5%이상 상승여부 삭제 

데이터 전처리

In [3]:
scaler = MinMaxScaler()
stock_label = scaler.fit_transform(filter_label.values.reshape(-1, 1))

count_zeros = np.sum(stock_label == 0)
count_ones = np.sum(stock_label == 1)
total_samples = len(stock_label)

ratio_zeros = (count_zeros / total_samples).round(2)
ratio_ones = (count_ones / total_samples).round(2)

print("Ratio of zeros (0s):", ratio_zeros)
print("Ratio of ones (1s):", ratio_ones)

Ratio of zeros (0s): 0.96
Ratio of ones (1s): 0.04


데이터 분할

In [4]:
# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(filter_stock, stock_label, test_size=0.2, random_state=42)

under_sampler = RandomUnderSampler(sampling_strategy='auto' , random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)
X_test_resampled, y_test_resampled = under_sampler.fit_resample(X_test, y_test)

smote = SMOTE(sampling_strategy=0.7 ,random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)
X_test_resampled, y_test_resampled = smote.fit_resample(X_test,y_test)

count_zeros = np.sum(y_resampled == 0)
count_ones = np.sum(y_resampled == 1)
total_samples = len(y_resampled)

ratio_zeros = (count_zeros / total_samples).round(2)
ratio_ones = (count_ones / total_samples).round(2)

print("Ratio of zeros (0s):", ratio_zeros)
print("Ratio of ones (1s):", ratio_ones)

Ratio of zeros (0s): 0.5
Ratio of ones (1s): 0.5


GRU 모델 정의

In [5]:
# GRU 모델 정의
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.GRU(64, input_shape=(X_resampled.shape[1],1),kernel_initializer='glorot_uniform', recurrent_dropout=0.0 ,return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=False))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 26, 64)            12864     
                                                                 
 gru_1 (GRU)                 (None, 26, 64)            24960     
                                                                 
 gru_2 (GRU)                 (None, 26, 64)            24960     
                                                                 
 gru_3 (GRU)                 (None, 26, 64)            24960     
                                                                 
 gru_4 (GRU)                 (None, 26, 64)            24960     
                                                                 
 gru_5 (GRU)                 (None, 26, 64)            24960     
                                                                 
 gru_6 (GRU)                 (None, 26, 64)            2

모델 훈련

In [6]:
# 모델 훈련
model.fit(X_resampled,y_resampled,epochs=500,
validation_data=(X_test_resampled, y_test_resampled),callbacks=[early_stopping])
 
# 모델 저장
model.save("GRU_Model_8L_64_5%.h5")

Epoch 1/500
1585/1585 [==============================] - 204s 125ms/step - loss: 0.4835 - accuracy: 0.7713 - val_loss: 0.3147 - val_accuracy: 0.8797
Epoch 2/500
1585/1585 [==============================] - 203s 128ms/step - loss: 0.2991 - accuracy: 0.8745 - val_loss: 0.2752 - val_accuracy: 0.8811
Epoch 3/500
1585/1585 [==============================] - 192s 121ms/step - loss: 0.2906 - accuracy: 0.8779 - val_loss: 0.3084 - val_accuracy: 0.8662
Epoch 4/500
1585/1585 [==============================] - 176s 111ms/step - loss: 0.2816 - accuracy: 0.8805 - val_loss: 0.2796 - val_accuracy: 0.8774
Epoch 5/500
1585/1585 [==============================] - 187s 118ms/step - loss: 0.2767 - accuracy: 0.8834 - val_loss: 0.2694 - val_accuracy: 0.8844
Epoch 6/500
1585/1585 [==============================] - 199s 125ms/step - loss: 0.2650 - accuracy: 0.8891 - val_loss: 0.2394 - val_accuracy: 0.9004
Epoch 7/500
1585/1585 [==============================] - 198s 125ms/step - loss: 0.2616 - accuracy: 0.8909

/Users/moon/anaconda3/envs/ML/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


모델 평가

In [7]:
# 모델 로드
loaded_model = tf.keras.models.load_model("GRU_Model_8L_64_5%.h5")

# 모델을 사용하여 주가 상승 여부 예측
test = loaded_model.predict(X_test_resampled)

8165/8165 [==============================] - 109s 13ms/step


In [8]:
y_pred_binary = (test > 0.5).astype(int)

accuracy = accuracy_score(y_test_resampled, y_pred_binary)
conf_matrix = confusion_matrix(y_test_resampled, y_pred_binary)
classification_rep = classification_report(y_test_resampled, y_pred_binary)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8978731059083793
Confusion Matrix:
[[136435  17255]
 [  9428  98155]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.89      0.91    153690
         1.0       0.85      0.91      0.88    107583

    accuracy                           0.90    261273
   macro avg       0.89      0.90      0.90    261273
weighted avg       0.90      0.90      0.90    261273



In [9]:
# 예측 결과를 이진값(0 또는 1)으로 변환
binary_predictions = (test > 0.5).astype(int)

# 예측 결과 출력X_test_resampled, y_test_resampled
correct_predictions = np.equal(binary_predictions, y_test_resampled)  # 정확하게 예측한 경우 True, 그렇지 않으면 False
accuracy = np.mean(correct_predictions)  # 정확도 계산

print(f"정확도: {accuracy * 100:.2f}%")

정확도: 51.03%
